In [1]:
import azureml.core
azureml.core.VERSION

Out[ 1 ]: '1.0.15'

In [2]:
workspace_name = 'AMLWorkspace1'
subscription_id = 'f78802b9-4646-4cf6-b7f1-6b2ee63219f4'
resource_group = 'ADBDevRg'

In [3]:
import os
import shutil
from azureml.core.workspace import Workspace

ws = Workspace(
    workspace_name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
aml_config = 'aml_config'
if os.path.isfile(aml_config) or os.path.isdir(aml_config):
    shutil.rmtree(aml_config)
ws.write_config()

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CLK5248F9 to authenticate.
Failed to authenticate '{'tenant_id': '0d993ad3-fa73-421a-b129-1fe5590103f3', 'id': '/tenants/0d993ad3-fa73-421a-b129-1fe5590103f3', 'additional_properties': {}}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: 2e601623-378f-41f7-a72e-333940089700\r\nCorrelation ID: 0397ab43-bba4-43e9-9d61-be55ce5c0a26\r\nTimestamp: 2019-02-25 12:12:14Z","error_codes":[50076],"timestamp":"2019-02-25 12:12:14Z","trace_id":"2e601623-378f-41f7-a72e-333940089700","correlation_id":"0397ab43-bba4-43e9-9d61-be55ce5c0a26","suberror":"basic_action"}'
Interactive authentication successfully completed.
Wrote the config file config.json to: /databricks/driver/aml_config/config.json

In [4]:
%sh
cat /databricks/driver/aml_config/config.json

{
 "subscription_id": "f78802b9-4646-4cf6-b7f1-6b2ee63219f4",
 "workspace_name": "AMLWorkspace1",
 "resource_group": "ADBDevRg"
}

In [5]:
ws.get_details()

Out[ 5 ]: 
{'id': '/subscriptions/f78802b9-4646-4cf6-b7f1-6b2ee63219f4/resourceGroups/ADBDevRg/providers/Microsoft.MachineLearningServices/workspaces/AMLWorkspace1',
 'containerRegistry': '/subscriptions/f78802b9-4646-4cf6-b7f1-6b2ee63219f4/resourcegroups/adbdevrg/providers/microsoft.containerregistry/registries/amlworkspace19971032709',
 'identityTenantId': 'd6397071-8e3e-45d2-a2d6-36698acf0fea',
 'keyVault': '/subscriptions/f78802b9-4646-4cf6-b7f1-6b2ee63219f4/resourcegroups/adbdevrg/providers/microsoft.keyvault/vaults/amlworkspace19128680434',
 'description': '',
 'applicationInsights': '/subscriptions/f78802b9-4646-4cf6-b7f1-6b2ee63219f4/resourcegroups/adbdevrg/providers/microsoft.insights/components/amlworkspace16642918728',
 'identityType': 'SystemAssigned',
 'identityPrincipalId': '92cd4f0c-30a8-404b-8c2d-4d671f9502b8',
 'storageAccount': '/subscriptions/f78802b9-4646-4cf6-b7f1-6b2ee63219f4/resourcegroups/adbdevrg/providers/microsoft.storage/storageaccounts/amlworkspace12956791781',
 'workspaceid': 'da471efe-9ff5-4bc8-b516-10e14b6515e6',
 'creationTime': '2019-01-29T23:33:53.8226405+00:00',
 'name': 'AMLWorkspace1',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'location': 'westeurope',
 'friendlyName': ''}

In [6]:
#persist the config file to dbfs so that it can be used for the other notebooks.
aml_config_local = 'file:' + os.getcwd() + '/' + aml_config
aml_config_dbfs = '/dbfs/' + 'aml_config'

if os.path.isfile(aml_config_dbfs) or os.path.isdir(aml_config_dbfs):
    shutil.rmtree(aml_config_dbfs)
    #dbutils.fs.rm(aml_config, recurse=True)

dbutils.fs.cp(aml_config_local, aml_config, recurse=True)

Out[ 6 ]: True

In [7]:
import os
#import urllib
#import pandas as pd

from pyspark.ml import PipelineModel

In [8]:
##NOTE: service deployment always gets the model from the current working dir. 
model_name = "modelMain"
model_path_dbfs = "/modelMain/"#os.path.join("/dbfs/models", model_name)
model_path_local = "file:" + os.getcwd() + "/" + model_name + "/"

print("copy model from dbfs {} to local {}".format(model_path_dbfs, model_path_local))
dbutils.fs.cp(model_path_dbfs, model_path_local, recurse=True)

copy model from dbfs /modelMain/ to local file:/databricks/driver/modelMain/
 Out[ 8 ]: True

In [9]:
import azureml.core
from azureml.core.workspace import Workspace

#get the config file from dbfs
aml_config = '/aml_config'
dbutils.fs.cp(aml_config, 'file:'+os.getcwd()+aml_config, recurse=True)

ws = Workspace.from_config()

Found the config file in: /databricks/driver/aml_config/config.json

In [10]:
#Register the model
from azureml.core.model import Model
mymodel = Model.register(model_path = model_name, # this points to a local file or folder in the current working dir
                       model_name = model_name, # this is the name the model is registered with                 
                       description = "Taxi Fare Prediction Model",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model modelMain
modelMain Taxi Fare Prediction Model 3

In [11]:
%%writefile score_sparkml.py

import json

def init():
    try:
        # One-time initialization of PySpark and predictive model
        import pyspark
        from pyspark.ml import PipelineModel
        from azureml.core.model import Model
        from mmlspark import LightGBMRegressor
        global trainedModel
        global spark
        
        spark = pyspark.sql.SparkSession.builder.appName("Scoring").getOrCreate()
      
        model_name = "modelMain" 
        
        model_path = Model.get_model_path(model_name)

        trainedModel = PipelineModel.load(model_path)

    except Exception as e:
        json.dumps("Exception in init: " + str(e))
        trainedModel = e

def run(input_df):
    response = ''    

    if isinstance(trainedModel, Exception):
        return json.dumps({"Exception":trainedModel})

    try:
#         print("received: " + input_df)
        
        sc = spark.sparkContext
      
        # Set inferSchema=true to prevent the float values from being seen as strings
        # which can later cause the VectorAssembler to throw an error: 'Data type StringType is not supported.'
        df = spark.read.option("inferSchema", "true").json(sc.parallelize([input_df]))
      
        #Get prediction results for the dataframe
        score = trainedModel.transform(df)
        predictions = score.collect()
        
        #Get each scored result (prediction and confidence)
        preds = [{"prediction":str(result['prediction'])} for result in predictions]
        
        response = preds[0]
        
#         print("response: " + str(response))
        
    except Exception as e:
        print("Exception in run: " + str(e))
        return json.dumps(str(e))

    # Return results
    return json.dumps(response)

Writing score_sparkml.py

Test the scoring script locally and confirm that it works as desired.

In [13]:
from mmlspark import LightGBMRegressor
model_name = "modelMain"         
model_path = Model.get_model_path(model_name)
trainedModel = PipelineModel.load(model_path)

In [14]:
dfx = spark.read.option("inferSchema", "true").json(sc.parallelize([{"pickup_longitude":-73.9667739868164, "pickup_latitude":40.75720977783203,
    "dropoff_longitude":-73.98005676269531, "dropoff_latitude":40.7390251159668, "passenger_count":1, "year":2015, "month":4, "day":1, "hour":18, "trip_distance_km":5}]))

In [15]:
import json
score = trainedModel.transform(dfx)
predictions = score.collect()
preds = [{"prediction":str(result['prediction'])} for result in predictions]
json.dumps(preds[0])

Out[ 14 ]: '{"prediction": "16.87123068237608"}'

In [16]:
# Create two records for testing the prediction
test_input1 = {"pickup_longitude":-73.9667739868164, "pickup_latitude":40.75720977783203,
    "dropoff_longitude":-73.98005676269531, "dropoff_latitude":40.7390251159668, "passenger_count":1, "year":2015, "month":4, "day":1, "hour":18, "trip_distance_km":5}

test_input2 = {"pickup_longitude":-73.9667739868164, "pickup_latitude":40.75720977783203,
    "dropoff_longitude":-73.98005676269531, "dropoff_latitude":40.7390251159668, "passenger_count":4, "year":2019, "month":1, "day":23, "hour":11, "trip_distance_km":8}

# test init() in local notebook
init()

# package the inputs into a JSON string and test run() in local notebook
test_inputs = [test_input1, test_input2] 
json_str_test_inputs = json.dumps(test_inputs)
run(json_str_test_inputs)

Out[ 15 ]: '{"prediction": "16.87123068237608"}'

In [17]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])
# myacienv = CondaDependencies()
myacienv.save_to_file(".", "mydeployenv.yml")

# with open("mydeployenv.yml","w") as f:
#     f.write(myacienv.serialize_to_string())

Out[ 16 ]: 'mydeployenv.yml'

In [18]:
myacienv.serialize_to_string()

Out[ 17 ]: '# Conda environment specification. The dependencies defined in this file will\r\n# be automatically provisioned for runs with userManagedDependencies=False.\r\n\n# Details about the Conda environment file format:\r\n# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually\r\n\nname: project_environment\ndependencies:\n # The python interpreter version.\r\n # Currently Azure ML only supports 3.5.2 and later.\r\n- python=3.6.2\n\n- pip:\n - azureml-defaults==1.0.15\n- scikit-learn\n- numpy\n- pandas\n'

In [19]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Taxi Fare Prediction'}, 
    description = 'Predicts a taxi fare.',
    properties = {'dns-name-label':'taxipr'})

In [20]:
service_name = "sparkmlservicedb09"
runtime = "spark-py" #"python" #
driver_file = "score_sparkml.py"
conda_file = "mydeployenv.yml"

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

In [21]:
webservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name, 
  deployment_config=aci_config,
  models = [mymodel], 
  image_config=image_config, 
  )

webservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image sparkmlservicedb09:1, operation "Succeeded"
Creating service
Running.................................................
SucceededACI service creation operation finished, operation "Succeeded"

In [22]:
def get_area(radius):
    request = json.dumps({"radius": 12})
    response = webservice.run(input_data = json_str_test_inputs)
    return json.loads(response)

get_area(radius=(0,10))

--------------------------------------------------------------------------- 
 WebserviceException Traceback (most recent call last)
 <command-2858132613738402> in <module> () 
 6 return json . loads ( response ) 
 7 
 ----> 8 get_area ( radius = ( 0 , 10 ) ) 

 <command-2858132613738402> in get_area (radius) 
 3 def get_area ( radius ) : 
 4 request = json . dumps ( { "radius" : 12 } ) 
 ----> 5 response = webservice . run ( input_data = json_str_test_inputs ) 
 6 return json . loads ( response ) 
 7 

 /databricks/python/lib/python3.5/site-packages/azureml/core/webservice/aci.py in run (self, input_data) 
 215 'Response Code: {}\n' 
 216 'Headers: {}\n' 
 --> 217 'Content: {}'.format(resp.status_code, resp.headers, resp.content))
 218 
 219 def update(self, image=None, tags=None, properties=None, description=None, auth_enabled=None, ssl_enabled=None,

 WebserviceException : Received bad response from service:
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '56', 'Date': 'Wed, 23 Jan 2019 16:28:55 GMT', 'X-Ms-Request-Id': '695d7f42-d938-4d7d-aa34-b7dfe1430b5b', 'X-Ms-Run-Function-Failed': 'True', 'Server': 'nginx/1.10.3 (Ubuntu)', 'Content-Type': 'text/html; charset=utf-8'}
Content: b"Object of type 'AttributeError' is not JSON serializable"

In [23]:
# webservice.run(input_data = json.dumps(test_input1))

import requests
headers = {'Content-Type':'application/json'}

# resp = requests.post(webservice.scoring_uri, "{\"data\": [" + json.dumps(test_input2) + "]}", headers=headers)
resp = requests.post(webservice.scoring_uri, json.dumps(test_input1), headers=headers)
print("POST to url", webservice.scoring_uri)
#print("input data:", input_data)
print("prediction:", resp.text)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1582041378449999> in <module> () 
 5 
 6 # resp = requests.post(webservice.scoring_uri, "{\"data\": [" + json.dumps(test_input2) + "]}", headers=headers) 
 ----> 7 resp = requests . post ( webservice . scoring_uri , json . dumps ( test_input1 ) , headers = headers ) 
 8 print ( "POST to url" , webservice . scoring_uri ) 
 9 #print("input data:", input_data) 

 NameError : name 'webservice' is not defined

In [24]:
test_input2 = {"pickup_longitude":-73.9667739868164, "pickup_latitude":40.75720977783203,
    "dropoff_longitude":-73.98005676269531, "dropoff_latitude":40.7390251159668, "passenger_count":4, "year":2019, "month":1, "day":23, "hour":11, "trip_distance_km":8}
json.dumps(test_input2)

Out[ 50 ]: '{"dropoff_latitude": 40.7390251159668, "pickup_longitude": -73.9667739868164, "hour": 11, "year": 2019, "passenger_count": 4, "month": 1, "trip_distance_km": 8, "day": 23, "pickup_latitude": 40.75720977783203, "dropoff_longitude": -73.98005676269531}'

In [25]:
json.dumps(test_input2)

Out[ 88 ]: '{"pickup_longitude": -73.9667739868164, "dropoff_longitude": -73.98005676269531, "hour": 11, "passenger_count": 4, "pickup_latitude": 40.75720977783203, "dropoff_latitude": 40.7390251159668, "day": 23, "trip_distance_km": 8, "month": 1, "year": 2019}'

In [26]:
json_str_test_inputs

Out[ 28 ]: '[{"hour": 18, "trip_distance_km": 5, "pickup_latitude": 40.75720977783203, "month": 4, "passenger_count": 1, "dropoff_longitude": -73.98005676269531, "day": 1, "dropoff_latitude": 40.7390251159668, "year": 2015, "pickup_longitude": -73.9667739868164}, {"hour": 11, "trip_distance_km": 8, "pickup_latitude": 40.75720977783203, "month": 1, "passenger_count": 4, "dropoff_longitude": -73.98005676269531, "day": 23, "dropoff_latitude": 40.7390251159668, "year": 2019, "pickup_longitude": -73.9667739868164}]'